In [1]:
import json
import pandas as pd
import numpy as np
import requests
import psycopg2
import os
import sqlalchemy
import tweepy

In [2]:
consumer_key = "tLlNyxv25Z4P6iDpNYohcnvQP"
consumer_secret = "NuNIcft96X61AuAIxvEUdH3AOjenRf86V2TVClw2iyclila9Kn"
access_token = "1501850566456193027-XeRgm19kD8EdNqt0OXxIdw5F2Cj0Fx"
access_token_secret = "dMM9yHiaCU8kehKNiJvKtF9dkgRgNupcX63Dcf4f8GH2l"

In [3]:
# Database credentials
ENDPOINT = 'datalaketwitter.c16p7bc6xhi2.us-east-1.rds.amazonaws.com'
DB_NAME = 'datalakeTwitter'
USERNAME = 'nadine'
PASSWORD = '30101995'

In [4]:
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 

In [13]:
# fetching tweets from the api
tweet_list_tasty_history = []
#result = api.user_timeline(screen_name='tasty')
for tweets in tweepy.Cursor(api.user_timeline, screen_name='tasty').items():
    tweet_list_tasty_history.append([tweets.text,
                             tweets.created_at.date(),
                             tweets.favorite_count,
                             tweets.retweet_count,
                             tweets.entities])

In [14]:
# putting the data into a pandas dataframe and adjusting the type of the entities

tasty_twitter_history = pd.DataFrame(tweet_list_tasty_history, columns=["tweet", "created", "favorites", "count",'additional_information'])

tasty_twitter_history['additional_information'] = tasty_twitter_history['additional_information'].apply(json.dumps)

In [15]:
#check if the data is displayed correctly
display(tasty_twitter_history)

,tweet,created,favorites,count,additional_information
0,"""I had the biggest gnarliest nail clipping in ...",2022-04-21,6,1,"{""hashtags"": [], ""symbols"": [], ""user_mentions..."
1,A Simple Veggie Curry changes my mood instantl...,2022-04-21,47,14,"{""hashtags"": [], ""symbols"": [], ""user_mentions..."
2,5 Cleaning Hacks That You Need In Your Life 😍 ...,2022-04-21,57,18,"{""hashtags"": [], ""symbols"": [], ""user_mentions..."
3,"Forget stuffed crust, give me all crust. https...",2022-04-21,11,3,"{""hashtags"": [], ""symbols"": [], ""user_mentions..."
4,What is the ultimate fruity dessert? While you...,2022-04-21,40,11,"{""hashtags"": [], ""symbols"": [], ""user_mentions..."
...,...,...,...,...,...
3245,🦄🍬🌈 https://t.co/QxXOep3qzz,2022-01-14,210,34,"{""hashtags"": [], ""symbols"": [], ""user_mentions..."
3246,RT @BuzzFeedVideo: 🥞+ 🐶 = pupcakes! https://t....,2022-01-14,0,159,"{""hashtags"": [], ""symbols"": [], ""user_mentions..."
3247,6-Ingredient No-Bake Caramel Tarts to bring to...,2022-01-14,307,52,"{""hashtags"": [], ""symbols"": [], ""user_mentions..."
3248,These Cheesecake bites came straight from heav...,2022-01-14,172,38,"{""hashtags"": [], ""symbols"": [], ""user_mentions..."


In [16]:
#try to make a connection to the database
try:
    print("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))
    conn = psycopg2.connect("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))
    
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)
        
try:
    cur = conn.cursor()
    
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)
        
# Create sqlalchemy engine
engine = sqlalchemy.create_engine("postgresql://{}:{}@{}:5432/{}".format(USERNAME,PASSWORD,ENDPOINT,DB_NAME))

host=datalaketwitter.c16p7bc6xhi2.us-east-1.rds.amazonaws.com dbname=datalakeTwitter user=nadine password=30101995


In [ ]:
#creating table if not existent
conn.set_session(autocommit=True)
cur.execute("CREATE TABLE IF NOT EXISTS tasty_twitter_history (tweet TEXT, created DATE, favorites INT, count INT, additional_information jsonb);")
    

In [ ]:
tasty_twitter_history.to_sql('tasty_twitter_history', engine, if_exists = 'append', index = False)

In [ ]:
cur.close()
conn.close()